# Franka Robot Grasping Tutorial: With and Without Spinning Pads

This tutorial demonstrates a comparison between two Franka Emika Panda robots—one with spinning pads and one without—performing the same grasping task. The spinning pads reduce the torsional friction about the normal vector of the contact between robot's finger and the object. 
"The implementation for friction in genesis is not as advanced that in Mujoco. There is only 1 scalar parameter in Genesis, and condim cannot be changed (only condim = 3 is implemented). There is no torsional friction at the moment. The torsional friction in genesis is the result of multi-contact friction."


The standard robot uses joints 0-8, with 0-6 for arm movement and 7-8 for the gripper. The modified robot includes spinning pads at joints 9-10, which can help provide rotational slippage at the contact point

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/15/25 15:14:41.108 23566] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.destroy()
gs.init(backend=gs.gpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [15:14:44] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [15:14:44] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [15:14:44] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [15:14:45] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [15:14:45] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [15:14:45] [INFO] Scene <243a364> created.


## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. Two Franka Emika Panda robots:
   - One standard robot at position (0,2,0)
   - One with spinning pads and virtual finger at position (0,0,0)
3. Two cubes for each robot
4. A target entity for visualization

The key difference between the robots is that the standard Franka `(franka_MJCF)` uses only the arm and finger joints (joints 0-8), while the other version has additional spinning pads (joints 9-10) that cause rotational slippage using extra hinge DoFs.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
                   
)
cube_size = (0.1, 0.07, 0.25)
cube_pos = [(0.7, 0, 0.05), (0.7, 2.0, 0.05)]

cube, cube2 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
        euler = (0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

[Genesis] [15:14:46] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <c14fbb8>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [15:14:46] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <5bc8299>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [15:14:47] [WARNING] (MJCF) Tendon not supported
[Genesis] [15:14:47] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [15:14:47] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [15:14:47] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [15:14:47] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [15:14:47] [INFO] Appl

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_hand`: Joint 9 at the middle of the finger for rotational slippage (only on the `franka`)
- `spinning_pads`: Joints 9-10 for the spinning contact pads (only on the `franka`)

In [4]:
n_envs = 0
scene.build()


[Genesis] [15:14:51] [INFO] Building scene <243a364>...


[Genesis] [15:14:52] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [15:14:52] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/15/25 15:14:54.079 23566] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [15:15:00] [INFO] Compiling simulation kernels...
[Genesis] [15:18:44] [INFO] Building visualizer...
[Genesis] [15:18:48] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
for joint in franka.joints:
    print(joint.name, joint.idx_local)
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
virtual_finger = 9
spinning_pads = np.arange(10, 12)

joint1 0
joint2 1
joint3 2
joint4 3
joint5 4
joint6 5
joint7 6
finger_joint1 7
finger_joint2 8
virtual_finger_joint 9
left_pad_hinge 10
right_pad_hinge 11


## Initial Robot Positioning

Reset the scene and set both robots to the same initial joint configuration. We'll use this "ready pose" as a starting position for our task. Notice that the standard robot has 9 DOFs (degrees of freedom), while the modified version can have 1 dof extra for virtual_finger and two extras for spinning_pads

In [6]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

# set joints position
franka.ready_qpos = np.append(qr, [0, 0, 0])
franka.set_qpos(franka.ready_qpos)
# position control
franka.control_dofs_position(franka.ready_qpos)

franka_MJCF.ready_qpos = qr
franka_MJCF.set_qpos(franka_MJCF.ready_qpos)
franka_MJCF.control_dofs_position(franka_MJCF.ready_qpos)

end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')

end_effector_MJCF = franka_MJCF.get_link('hand')


target_entity.set_pos(virtual_finger.get_pos())
target_entity.set_quat(virtual_finger.get_quat())


scene.step()

[Genesis] [15:18:57] [INFO] Resetting Scene <243a364>.


## Pre-grasp Positioning

Now we'll move both robots to a position above the cube. We'll use inverse kinematics to calculate the joint positions needed to place the end-effector at the desired position and orientation. 


In [17]:

# get the end-effector link
l, w, h, = adapter.get_size(cube)
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[2]/2 + 0.02, 0, 0.1 + cube_size[0]/2 - 0.02]) # 0.1 m above the grasp point
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = above_box,
    rot_mask = [False, False, True]
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.1034]), # 0.1034 m is the distance between hand and fingers
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos_MJCF[fingers_dof] = 0.04
qpos[fingers_dof] = 0.04
print(qpos)


tensor([ 6.5570e-05,  3.1011e-01, -3.8616e-05, -2.0730e+00, -2.1062e-04,
         2.4214e+00,  7.8540e-01,  4.0000e-02,  4.0000e-02, -4.1957e-10],
       device='cuda:0')


In [18]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

[Genesis] [17:31:52] [INFO] Path solution found successfully.
[Genesis] [17:31:52] [INFO] Path solution found successfully.


## Path Planning

Generate a smooth trajectory for each robot to reach the pre-grasp position.

In [19]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


## Gripper Positioning

Going down to align fingers with grasp point

(array([ 5.95016677e-01, -1.42341375e-10,  1.79892239e-01]),
 array([ 5.95016677e-01, -1.42341375e-10,  7.98922393e-02]))

In [24]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = side_box,
    rot_mask = [False, False, True]
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.1034]),
    quat = np.array([0, 1, 0, 0]),
)


## Reaching for the Object

Calculate and execute a path to move from the pre-grasp position down to the actual grasp position. Both robots move their end-effectors to the side of their respective cubes.


In [48]:
end_effector_dq = adapter.forward_kinematics(franka, end_effector, output_type='dq')
cube_bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(cube_bbox)
q, s_axis = qs[1], s_axes[1]

In [49]:

trajectory_franka = planner.generate_screw_trajectory(initial_pose=end_effector_dq, q=q, s_axis=s_axis, output_type='pq')

In [50]:
trajectory_franka

array([[ 5.95393598e-01, -3.39945145e-05,  1.75905704e-01, ...,
         9.99652239e-01,  2.96501394e-04, -2.63672591e-02],
       [ 5.95393701e-01, -3.39945145e-05,  1.75905839e-01, ...,
         9.99652232e-01,  2.96501477e-04, -2.63675514e-02],
       [ 5.95394417e-01, -3.39945145e-05,  1.75906773e-01, ...,
         9.99652178e-01,  2.96502054e-04, -2.63695853e-02],
       ...,
       [ 1.00104483e+00, -3.39945145e-05,  2.29528039e-01, ...,
         6.88218098e-01,  4.10042979e-04, -7.25503743e-01],
       [ 1.00104576e+00, -3.39945145e-05,  2.29527323e-01, ...,
         6.88216622e-01,  4.10042960e-04, -7.25505144e-01],
       [ 1.00104590e+00, -3.39945145e-05,  2.29527220e-01, ...,
         6.88216409e-01,  4.10042957e-04, -7.25505345e-01]],
      shape=(300, 7))

In [ ]:

for waypoint in trajectory_franka:
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [17:36:25] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [17:36:25] [INFO] Path solution found successfully.
[Genesis] [17:36:25] [INFO] Ingoring collision pairs already active for starting pos.
[Genesis] [17:36:26] [INFO] Path solution found successfully.


In [26]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)
franka_MJCF.set_qpos(qpos_reach_MJCF)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[motors_dof], motors_dof)
franka_MJCF.control_dofs_force(np.array([-2, -2]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


## Grasping the Object

Now we'll close the grippers on both robots by applying force to the finger joints. This is where we can start observing differences:

1. The standard robot (without spinning pads) applies force only with its parallel-jaw gripper
2. The enhanced robot also activates its spinning pads, which distribute force more evenly across the object surface

The spinning pads reduce the chance of the object slipping by providing rolling contact rather than pure sliding contact.


In [ ]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-4], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

## Lifting the Object

With the object grasped, we now lift both cubes upward. Watch carefully for any differences in stability:

- The standard gripper may show slight object rotation or slippage during movement
- The gripper with spinning pads typically maintains a more stable grasp by adapting to small forces with rolling action


In [ ]:
# move backward
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box + np.array([-0.1, 0.0, 0.1]),
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([-0.1, 2.0, 0.1]),
    quat = np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-4], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

## Transporting the Object

Finally, we move the robots backward while still grasping their cubes. This lateral movement introduces additional forces that can challenge grasp stability.

The differences between the two gripping methods should be most apparent during this phase:
- The standard gripper relies solely on friction to prevent slipping
- The gripper with spinning pads allows controlled rolling at the contact points, reducing the build-up of lateral forces

This demonstrates why spinning pads can be advantageous for delicate object manipulation tasks where maintaining precise grasp stability is critical.


## Conclusion

This tutorial demonstrated the difference between standard Franka grippers and those equipped with spinning pads. Key observations:

1. **Setup Complexity**: The spinning pad configuration requires additional DOFs (9-10) but provides enhanced manipulation capabilities
2. **Grasp Stability**: Spinning pads can improve stability by providing rolling contact instead of sliding contact
3. **Object Control**: During transport, the spinning pads help maintain more precise control of the object

For applications requiring precise manipulation or handling of delicate objects, the spinning pad configuration offers significant advantages despite the added complexity.

**Next Steps**:
- Try manipulating objects of different shapes and materials
- Experiment with different spinning pad velocities
- Implement closed-loop control using force feedback from the gripper
